In [1]:
### Plotting bettis

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using JLD
using Plots
using ColorSchemes


println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-172570514159908403\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6332371948229628424\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-172570514159908403\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


importing packages...
packages and functions imported
Elapsed time = 20.738147020339966 seconds 
 


In [2]:
### Set parameters

# Parameters for all graphs
const NNODES = 70
const MAXDIM = 3    # Maximum persistent homology dimension
const NREPS = 50
const DATESTRING = "082520"


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis.jld",x), readdir(read_dir))
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end


# Locate the nametags
nametags = []
for betti_file in betti_files
    tag = split(split(betti_file, "$(DATESTRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


Located the following graph files:
DP_50_70_3_082520_backward_bettis.jld
DP_50_70_3_082520_forward_bettis.jld
DP_50_70_3_082520_thresh005_edge121_threshold_bettis.jld
DP_50_70_3_082520_thresh015_edge363_threshold_bettis.jld
DP_50_70_3_082520_thresh01_edge242_threshold_bettis.jld
DP_50_70_3_082520_thresh02_edge483_threshold_bettis.jld
DP_50_70_3_082520_thresh03_edge725_threshold_bettis.jld
DP_50_70_3_082520_thresh04_edge966_threshold_bettis.jld
DP_50_70_3_082520_thresh05_edge1208_threshold_bettis.jld
DP_50_70_3_082520_thresh06_edge1449_threshold_bettis.jld
IID_50_70_082520_backward_bettis.jld
IID_50_70_082520_forward_bettis.jld
IID_50_70_082520_thresh005_edge121_threshold_bettis.jld
IID_50_70_082520_thresh015_edge363_threshold_bettis.jld
IID_50_70_082520_thresh01_edge242_threshold_bettis.jld
IID_50_70_082520_thresh02_edge483_threshold_bettis.jld
IID_50_70_082520_thresh03_edge725_threshold_bettis.jld
IID_50_70_082520_thresh04_edge966_threshold_bettis.jld
IID_50_70_082520_thresh05_edge120

10-element Array{Any,1}:
 "backward"                   
 "forward"                    
 "thresh005_edge121_threshold"
 "thresh015_edge363_threshold"
 "thresh01_edge242_threshold" 
 "thresh02_edge483_threshold" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_threshold"
 "thresh06_edge1449_threshold"

In [6]:


for nametag in nametags
    
    ## Create one large plot with all of the betti curves
    betti_files_nametag = filter(x -> occursin("$(nametag)",x), betti_files)
    model_names = [split(betti_file_nametag,"_")[1] for betti_file_nametag in betti_files_nametag]
    
    nModels = length(betti_files_nametag)
    nEdges = binomial(NNODES, 2)
    bettisAll = zeros(NREPS, nEdges, MAXDIM, nModels)
    
    for (i,betti_file_nametag) in enumerate(betti_files_nametag)

        # Read in Betti curves
        betti_dict = load("$(read_dir)/$(betti_file_nametag)")
        bettisAll[:,:,:, i] = betti_dict["bettisArray"]
        println(betti_file_nametag)

    end

    bettisAllAvg = dropdims(mean(bettisAll, dims=1), dims=1)
    println("Size bettisAllAvg is $(size(bettisAllAvg))")

    # Finally plot
    p1a = plot(bettisAllAvg[:,1,:], layout = 12, size = (1000,800), title = ["$(model_names[i])" for j=1:1, i=1:nModels])
    for d=2:MAXDIM
        plot!(bettisAllAvg[:,d,:],legend = nothing, xlabel = "Edges added", ylabel="Beta_k")

    end
    
    # If the file is of threshold type, add the threshold line
    if occursin("threshold", nametag)
        
        # Get threshold edge
        thresh_string = split(split(nametag,"edge")[2],"_")[1]
        threshold_edge = parse(Int, thresh_string)
        println("processing threshold edge $(threshold_edge)")
        
        # Plot the vertical line
        vline!(transpose([threshold_edge for i=1:nModels]), color=:gray)
    end


    savefig("../figures/all_bettis_$(nametag).pdf")
end






DP_50_70_3_082520_backward_bettis.jld
65.0
IID_50_70_082520_backward_bettis.jld
472.0
RG_50_70_3_082520_backward_bettis.jld
1888.0
RL_50_70_082520_backward_bettis.jld
14951.0
assoc_700_100_100_20_20_09_05_082520_backward_bettis.jld
982.0
coreperiph_700_100_100_20_20_09_05_082520_backward_bettis.jld
182.0
cosineGeometric_50_70_3_082520_backward_bettis.jld
8.0
disassort_700_100_100_20_20_09_05_082520_backward_bettis.jld
352.0
discreteUniformConf_50_70_0_1000_082520_backward_bettis.jld
73.0
geometricConf_500_700_001_1000_082520_backward_bettis.jld
64.0
Size bettisAllAvg is (2415, 3, 10)
DP_50_70_3_082520_forward_bettis.jld
1.0
IID_50_70_082520_forward_bettis.jld
473.0
RG_50_70_3_082520_forward_bettis.jld
15.0
RL_50_70_082520_forward_bettis.jld
13.0
assoc_700_100_100_20_20_09_05_082520_forward_bettis.jld
291.0
coreperiph_700_100_100_20_20_09_05_082520_forward_bettis.jld
213.0
cosineGeometric_50_70_3_082520_forward_bettis.jld
4596.0
disassort_700_100_100_20_20_09_05_082520_forward_bettis.jl

In [5]:
nametags

10-element Array{Any,1}:
 "backward"                   
 "forward"                    
 "thresh005_edge121_threshold"
 "thresh015_edge363_threshold"
 "thresh01_edge242_threshold" 
 "thresh02_edge483_threshold" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_threshold"
 "thresh06_edge1449_threshold"

In [11]:
a = []
append!(a,["two"])

1-element Array{Any,1}:
 "two"